In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project\\research'

In [3]:
os.chdir("../")

In [ ]:
%pwd

'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project'

Defining the Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: str
    data_path: str
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    gradient_accumulation_steps: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    per_device_eval_batch_size: int
        


Defining the configuration manager:

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    
    #Defining a constructor with following paramteres:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        #Obtaining the config from the file_path:
        self.config = read_yaml(config_filepath)
        
        #Obtaining the paramters from the file_path:
        self.params = read_yaml(params_filepath)

        #Creating a new artifact: root folder:
        create_directories([self.config.artifacts_root])

    
    #Defining the function for obtaining the model trainer config:
    def get_model_trainer_config(self) -> ModelTrainerConfig:

        #Obtaining the configuration:
        config = self.config.model_trainer

        #Obtaining the parameters:
        params = self.params.TrainingArguments

        #Creating a directory storing the configuration root directory:
        create_directories([config.root_dir])

        #Creating the model_trainer config:
        model_trainer_config = ModelTrainerConfig(

            #Setting up all the values:
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps=max(1,(params.eval_steps*((params.num_train_epochs//10) or 1))),
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
        )

        return model_trainer_config


Creating the components:

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\shiva\OneDrive\Desktop\Work\NLP\Projects\Text_Summarizer_Project\textSummarization\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-04 11:30:19,617: INFO: config: PyTorch version 2.6.0 available.]


In [9]:
class ModelTrainer:

    #Constructor:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    #A function to train the model:
    def train(self):

        # Setting up the device:
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Setting up the tokenizer:
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)

        #Loading the data:
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum_pt.remove_columns(
            [col for col in dataset_samsum_pt["train"].column_names if col not in ["input_ids", "attention_mask", "labels"]]
        )
        dataset_samsum_pt.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


        # Setting up the model:
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
    

        # Setting up the seq2seq2 Collator:
        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model_pegasus,
            label_pad_token_id=-100,
        )

        checkpoint_dir = os.path.join(self.config.root_dir, "checkpoints")
        os.makedirs(checkpoint_dir, exist_ok=True)

        last_checkpoint = None
        checkpoint_files = sorted(
            [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")],
            key=lambda x: int(x.split("-")[-1]) if x.split("-")[-1].isdigit() else 0
        )
        
        if checkpoint_files:
            last_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1])
            print(f"Resuming training from checkpoint: {last_checkpoint}")
        else:
            print("No checkpoint found, training from scratch...")
        

        #Setting up the training argumets: Reading the paramters from params.yaml:
        trainer_args = TrainingArguments(
            output_dir = os.path.join(self.config.root_dir,"checkpoints"),
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps=self.config.logging_steps*2, 
            evaluation_strategy="steps",
            eval_steps=self.config.eval_steps,
            save_strategy = "steps",
            save_steps = 25,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            save_total_limit=2,
            report_to = "none",
            load_best_model_at_end=True,
            remove_unused_columns=False
        )

        # Initialising the trainer:
        trainer = Trainer(
            model = model_pegasus,
            args = trainer_args,
            tokenizer = tokenizer,
            data_collator= seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset= dataset_samsum_pt["validation"])
        
        #Training the trainer:
        trainer.train(resume_from_checkpoint = last_checkpoint)

        #Saving the model:
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        #Saving the tokenizer:
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
    

In [10]:
import os
os.environ["OMP_NUM_THREADS"] = "4"  

Creating the pipeline:

In [1]:
try: 
    #Intialising the configuration manager:
    config = ConfigurationManager()

    #Obtaining the model configuration:
    model_trainer_config = config.get_model_trainer_config()

    #Applying the model_trainer_config into model_trainer object:
    model_trainer_config = ModelTrainer(config = model_trainer_config)

    #Calling the train function:
    model_trainer_config.train()

except Exception as e:
    raise e

NameError: name 'ConfigurationManager' is not defined